In [1]:
import pandas as pd
from random import randint
import time
import re
# Scraping through Chrome webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [2]:
# Starting URLs
centris = "https://www.centris.ca/en/properties~for-sale?view=Thumbnail"
duproprio = "https://duproprio.com/en/search/list?search=true&is_for_sale=1&with_builders=1&parent=1&pageNumber=1&sort=-published_at"

In [426]:
class Centris:
    """
    Accessing 
    
    Attr:
    self.url - starting url for scraping process
    self.data - pandas.DataFrame object containing scraped data
    self.driver - Chrome webdriver
    self.containers - List of web-elements containing information on listings
        - eg: link to listing, price, picture, address,...
    self.links_to_listings - List of web-elements, each containing the link to a listing
    """
     
    def __init__(self, url="https://www.centris.ca/en/houses~for-sale~lac-simon/11851081?view=Summary&uc=3"): 
        self.url = url
        self.data = pd.DataFrame({\
                        'title': None,\
                        'address': None,\
                        'price': None,\
                        'lat': None,\
                        'long': None,\
                        'descriptions': None,\
                        'neighbourhood_indicators': None,\
                        'demographics': None
                    }, index=[0])
        # Path to Chromedriver
        self.DRIVER_PATH = 'C:/webdriver/chromedriver.exe'
        self.driver = None
        # Verification for new DOM
        self.old_DOM = {\
                        "title" : [],\
                        "address": [],\
                        "price": [],\
                        "lat": [],\
                        "long": [],\
                        "descriptions": [],\
                        'neighbourhood_top': [],\
                        'neighbourhood_buttom': [],\
                        "demographics_buttons": [],\
                    }

    def append_data(self, title, address, price,\
            lat, long, descriptions, neighbourhood_indicators,\
            demographics):
        """Appends data to columns."""

        new_data = pd.DataFrame({\
                        'title': title,\
                        'address': address,\
                        'price': price,\
                        'lat': lat,\
                        'long': long,\
                        'descriptions': descriptions,\
                        'neighbourhood_indicators': neighbourhood_indicators,\
                        'demographics': demographics\
                    }, index=[0])
        self.data = self.data.append(new_data, ignore_index=True)
             
    def get_data(self):
        return self.data
        
    def start_driver(self):
        """
        Starts and returns Crome webdriver. 
        The page link in the url attribute 
        is opened in headless mode.
        """
        
        # Activate headless mode for fastest response
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-infobars"); # disabling infobars
        options.add_argument("--disable-extensions"); # disabling extensions
        options.add_argument("--disable-gpu"); # applicable to windows os only
        options.add_argument("--disable-dev-shm-usage"); # overcome limited resource problems
        options.add_argument("--no-sandbox"); # Bypass OS security model
        options.add_argument('--start-maximized') # open Browser in maximized mode
        options.add_argument('--incognito')

        # Start driver with url
        self.driver = webdriver.Chrome(executable_path=self.DRIVER_PATH)
        self.driver.get(self.url)

    def sort_listings(self):
        """Sorts listings in webdriver from newest to oldest."""
        
        # Click drop down menu
        drop_down = self.driver.find_element_by_xpath(\
                                    "//button[@id='dropdownSort']")
        drop_down.click()
        
        # Sort by most recent listings
        sort_by = self.driver.find_element_by_xpath("//a[@data-option-value='3']")
        sort_by.click()
    
    def goto_first_page(self):
        try:
            next_page = self.driver.find_element_by_xpath(\
                                        "//li[@class='goFirst']")
            next_page.click()
        except:
            print("goFirst button not available")
    
    def next_page(self):
        try:
            next_page = self.driver.find_element_by_xpath(\
                                        "//li[@class='next']")
            next_page.click()
            pass
        except:
            time.sleep(0.5)
            # Try again after waiting 0.5 sec.
            try:
                next_page = self.driver.find_element_by_xpath(\
                                            "//li[@class='next']")
                next_page.click()
                pass
            except:
                print("Next-page button not found!")
                
    def get_last_page(self):
        '''Returns page number of last page in browser.'''
        
        last_page = self.driver.find_element_by_xpath(\
                                    "//li[@class='pager-current']").text
        return int(last_page.split(" / ")[1].replace(",", ""))
    
    def refresh_page(self):
        "Refreshes current webdriver page."
        self.driver.refresh()
        # Wait until page fully loaded
        time.sleep(2)
                

                                                  
# Instantiate class object
centris = Centris()

The following functions need to be outside of the Class. wait_for_xpath() determined the approptiate time to call get_data(). Initially, both fuctions were part of the class object. It seems that after the get_data() call, the driver does not get updated within the class. This leads in some cases to old DOM's being accessed after the browser has already switched to the next page. To circumvent this issue, elements are called outside the class and tried until accessible. This allows the entire new DOM to be loaded before get_data() is called.

In [424]:
 def wait_for_xpath(xpath: str, old_element):
        """
        Wait until elements in new DOM are accessible.
        
        Returns:
        current_element - the element found in the new DOM at xpath
        """
        
        # Wait until DOM accessible
        time.sleep(0.2)
        centris_driver = centris.driver
        current_element = []
        try:
            current_element = centris_driver.find_elements_by_xpath(xpath)
        except:
            time.sleep(0.2)
            try:
                print("Second attempt loading element!")
                current_element = centris_driver.find_elements_by_xpath(xpath)
            except:
                time.sleep(0.15)
                try:
                    print("Third and last attempt loading element!")
                    current_element = centris_driver.find_elements_by_xpath(xpath)
                except:
                    print("TimeOutError: Unable to load element")
        
        # LOGGING --------------------------------
#         print("Previous:\n", old_element)
#         print("Current:\n", current_element)
#         print("="*50)
        
        # Ensure that the NEW rather than the previous DOM is active
        time_passed = 0
        while (current_element == old_element\
               and time_passed < 10):
            print("Waiting for new DOM...")
            time.sleep(0.15)
            time_passed += 0.15
            current_element = centris_driver.find_elements_by_xpath(xpath)
        return current_element

In [399]:
def scrape_description(old_DOM):
    """ Requires instantiated centris object. Scrapes and returns
    description data: Year build, price, Lot area, etc."""
    
    descriptions = wait_for_xpath("//div[@class='col-lg-12 description']",\
                                 old_DOM)
    #First three elements not relevant
    descriptions_list = descriptions[0].text.split("\n")[3:]
    
    #LOGGING------------------------
    print("DESCRIPTION:", descriptions_list)
    
    # Update old_DOM dictionary with new element for next verification
    centris.old_DOM['descriptions'] = descriptions
    
    return descriptions_list

In [614]:
def scrape_neighbourhood(old_DOM_top, old_DOM_buttom):
    """ Scrapes and returns a list of ratings 
    between 0-10 for a set of neighborhood indicators
    such as groceries, parks, noise, etc.)
    """
    driver = centris.driver
    
    # Elements from top of scrollable list
    neighbourhood_top = wait_for_xpath(\
                            "//div[@class='ll-list ps ps--active-y']",\
                            old_DOM_top)
    # Split into indicators and ranking values
    top = [x.text for x in neighbourhood_top][0].split("\n")
    
    # LOGGING----------------------
    print("Top neighbourhood:", top)
    
    # Scroll to buttom of list to load remaining elements
    scrollable_bar = driver.find_element_by_xpath(\
                                            "//div[@class='ps__thumb-y']")
    scrollable_bar.send_keys(Keys.PAGE_DOWN)
    scrollable_bar.click()
    
    # Elements from buttom of scrollable list
    neighbourhood_buttom = driver.find_elements_by_class_name('ll-list')
    
    # Split into indicators and ranking values
    buttom = [x.text for x in neighbourhood_buttom][0].split("\n")
    
    # LOGGING----------------------
    print("Buttom neighbourhood:", buttom)
    
    # Unite both lists by storing tuples of indicator names and corresponding values
    united_list = []
    list_length = len(top)
    for i in range(0, list_length, 2):
        united_list.append((top[i], top[i+1]))
        united_list.append((buttom[i], buttom[i+1]))
    
    # Create set of unique tuples
    neighbourhood_indicators = set(united_list)
    
    # LOGGING----------------------
    print("Number of neighborhood indicators: ", len(neighbourhood_indicators))
    print("UNITED:", united_list)
    print("SET:", neighbourhood_indicators)
    
    # Verify size and extract information as list
    # If size unexpected, refresh page and restart process
    if len(neighbourhood_indicators) < 8:
            centris.refresh_page()
            scrape_neighbourhood(old_DOM_top, old_DOM_buttom)

    # Update old_DOM dictionary with new elements for next verification
    centris.old_DOM['neighbourhood_top'] = neighbourhood_top
    centris.old_DOM['neighbourhood_buttom'] = neighbourhood_buttom

    return neighbourhood_indicators

In [628]:
def scrape_population():
    """Scrapes and returns population summary data (density, variation etc.)"""
    population_summaries =  centris.driver.find_element_by_id('info')
    population_summaries_list = population_summaries\
                        .text.split("\n")
    
    # LOGGING-----------------------
    print("Population:", population_summaries_list)
    
    return population_summaries_list

In [647]:
def scrape_demographics(old_DOM):
    """Scrapes and return demographic data found in a clickable list"""
    
    #Buttons to access demographics data (education, incomes, etc.)
    demographics_buttons = wait_for_xpath(\
                        "//div[@class='centrisSocioDemobutton']",\
                                                 old_DOM)

    # LOGGING------------------------
    # print("DEMO. BUTTONS:", demographics_buttons)

    # First entry on clickable demographics list (pre-selected)
    demographics = []

    # Click buttons to access next demogrpahics elements
    for button in demographics_buttons:
        try: 
            button.click()
        except: 
            print("Demogrphics button missing!")
            demographics.append(None)
            
        # Get and append data after button click
        demographic_data = centris.driver.find_element_by_class_name(\
                         "socioDemoLabel")
        demographics.append(demographic_data.text)
    
    # Split each demographic component into separate list
    # Example: splits "Occupation" data into -> ["Owners", "35%", "Renters", "65%"]
    demographics = [demo.split("\n") for demo in demographics]
    
    #LOGGING------------------------
    print("DEMO. DATA:", demographics)
    print("-"*50)
    
    # Update old_DOM dictionary with new elements for next verification
    centris.old_DOM['demographics_buttons'] = demographics_buttons
    
    return demographics

In [645]:
def get_data_from_centris():
        """
        Requires instantiate Centris object. Scrapes information from the
        webdriver and appends it to the Centris object.
        """
        driver = centris.driver
        old_DOM = centris.old_DOM
        
        # Data from headers
        print("Start scraping new page...")
        title = wait_for_xpath("//span[@data-id='PageTitle']", old_DOM['title'])
        address = wait_for_xpath("//h2[@itemprop='address']", old_DOM['address'])
        price = wait_for_xpath("//span[@itemprop='price']", old_DOM['price'])
        lat = wait_for_xpath("//meta[@itemprop='latitude']", old_DOM['lat'])
        long = wait_for_xpath("//meta[@itemprop='longitude']", old_DOM['long'])
        
        # Save elements as old DOM
        centris.old_DOM['title'] = title
        centris.old_DOM['address'] = address
        centris.old_DOM['lat'] = lat
        centris.old_DOM['long'] = long
        
        descriptions = scrape_description(old_DOM['descriptions'])
        neighbourhood_indicators_list = scrape_neighbourhood(old_DOM['neighbourhood_top'],\
                                                           old_DOM['neighbourhood_buttom'])
        population_summaries_list = scrape_population()
        demographics = scrape_demographics(old_DOM['demographics_buttons'])
                
        # Register data in dataframe
#         centris.append_data(
#             title[0].text,\
#             address[0].text,\
#             price[0].text,\
#             lat[0].get_attribute("content"),\
#             long[0].get_attribute("content"),\
#             descriptions,\
#             neighbourhood_indicators_list,\
#             demographics\
#         )
        
        # Return to top of page, to access next-page button
        body = driver.find_element_by_tag_name("body")
        for i in range(7):
            body.send_keys(Keys.PAGE_UP)

## Testing

In [527]:
# Test
start = time.time()
centris.start_driver()
centris.sort_listings()
print("Execution time:", time.time() - start)

Execution time: 9.49912691116333


Before running the next cell, search for the region(s) you want to scrape in the webdriver window.
This is not required but will substential limit run time and narrow results.

In [649]:
start = time.time() 
total_pages = centris.get_last_page() # Number of total listings
one_to_100 = range(1,100) # Used to print message after each 1% completion

print("Scraping initiated.")
print("Totla number of pages to scrap:", total_pages)
print("="*50)

for i in range(100):
    # -0.01 corrects for overshoots
    percent_complete = round(100*((i-0.01)/total_pages),1) # percent completed of scraping
    print("Page:", i+1)
    time_passed = 0 # to exit while loop after 10 seconds
    
    #Refresh every 20 pages to clear memory build-up
    if (i+1)%20 == 0:
        print("Refreshing page")
        print("-"*50)
        
        # Each refresh frees some memory. Four seem to work best.
        for i in range(4):
            centris.driver.refresh()
            # Give time to load fresh page
            time.sleep(0.3)
            # Extra time for last refresh
            # Ensures that DOM is fully loaded
            if i == 3:
                time.sleep(2)
            
    #Retrieve data    
    get_data_from_centris()
    
    # Short delay for chrome to respond to PAGE_UP command
    time.sleep(0.5)
    centris.next_page()
            
    # Print after every 1% mark
    if percent_complete in one_to_100:
        print(percent_complete, "%", "completed")
        print("Estimated remaining runtime:", round(total_pages*((execution_time/N)/(60*60)), 1), "hours")
        print("="*50)

execution_time = time.time() - start
print("Total runtime:", execution_time/(60*60), "hours")
centris.data

Scraping initiated.
Totla number of pages to scrap: 45119
Page: 1
Start scraping new page...
DESCRIPTION: ['Use of property', 'Residential only', 'Building style', 'Detached', 'Year built', '1987', 'Parking (total)', 'Driveway (8)', 'Number of units', 'Residential (4)', 'Residential units', '4 x 4 ½', 'Main unit', '4 rooms, 2 bedrooms, 1 bathroom', 'Potential gross revenue', '$32,676', '22']
Top neighbourhood: ['Quiet', '10', 'Car friendly', '9', 'Elementary Schools', '7', 'Parks', '6', 'Groceries', '6', 'Cafes', '5', 'Transit friendly', '4', 'Restaurants', '4']
Buttom neighbourhood: ['Transit friendly', '4', 'Restaurants', '4', 'Pedestrian friendly', '3', 'Nightlife', '3', 'Shopping', '3', 'Daycares', '2', 'High Schools', '1', 'Vibrant', '1']
Number of neighborhood indicators:  14
UNITED: [('Quiet', '10'), ('Transit friendly', '4'), ('Car friendly', '9'), ('Restaurants', '4'), ('Elementary Schools', '7'), ('Pedestrian friendly', '3'), ('Parks', '6'), ('Nightlife', '3'), ('Groceries', 

Page: 4
Start scraping new page...
DESCRIPTION: ['5 rooms', '2 bedrooms', '1 bathroom', 'Condominium type', 'Divided', 'Net area', '869 sqft', 'Year built', '1890', 'Additional features', 'Basement 6 feet or +', '92']
Top neighbourhood: ['Elementary Schools', '10', 'Pedestrian friendly', '10', 'Parks', '10', 'Groceries', '10', 'Quiet', '9', 'Restaurants', '9', 'High Schools', '8', 'Daycares', '8']
Buttom neighbourhood: ['Daycares', '8', 'Shopping', '8', 'Transit friendly', '7', 'Vibrant', '7', 'Nightlife', '6', 'Cafes', '6', 'Car friendly', '5', 'Cycling friendly', '5']
Number of neighborhood indicators:  15
UNITED: [('Elementary Schools', '10'), ('Daycares', '8'), ('Pedestrian friendly', '10'), ('Shopping', '8'), ('Parks', '10'), ('Transit friendly', '7'), ('Groceries', '10'), ('Vibrant', '7'), ('Quiet', '9'), ('Nightlife', '6'), ('Restaurants', '9'), ('Cafes', '6'), ('High Schools', '8'), ('Car friendly', '5'), ('Daycares', '8'), ('Cycling friendly', '5')]
SET: {('Shopping', '8'), ('

Page: 7
Start scraping new page...
DESCRIPTION: ['17 rooms', '4 bedrooms', '3 bathrooms and 1 powder room', 'Building style', 'Two or more storey, Detached', 'Year built', '1977', 'Lot area', '12,942 sqft', 'Parking (total)', 'Driveway (4)', 'Pool', 'Inground', 'Fireplace/Stove', 'Wood fireplace, Gas fireplace', 'Additional features', 'Basement 6 feet or +', '27']
Top neighbourhood: ['Quiet', '10', 'Car friendly', '10', 'Parks', '6', 'Groceries', '6', 'Transit friendly', '4', 'Greenery', '4', 'Elementary Schools', '4', 'Pedestrian friendly', '4']
Buttom neighbourhood: ['Restaurants', '4', 'Shopping', '4', 'Daycares', '2', 'Vibrant', '2', 'Cafes', '2', 'High Schools', '1', 'Nightlife', '1', 'Historic', '0']
Number of neighborhood indicators:  16
UNITED: [('Quiet', '10'), ('Restaurants', '4'), ('Car friendly', '10'), ('Shopping', '4'), ('Parks', '6'), ('Daycares', '2'), ('Groceries', '6'), ('Vibrant', '2'), ('Transit friendly', '4'), ('Cafes', '2'), ('Greenery', '4'), ('High Schools', '1

Page: 10
Start scraping new page...
Waiting for new DOM...
DESCRIPTION: ['7 rooms', '3 bedrooms', '1 bathroom', 'Condominium type', 'Divided', 'Net area', '1,085 sqft', 'Year built', '1905, Century', '93']
Top neighbourhood: ['Elementary Schools', '10', 'High Schools', '10', 'Pedestrian friendly', '10', 'Parks', '10', 'Groceries', '10', 'Vibrant', '10', 'Cycling friendly', '10', 'Nightlife', '10']
Buttom neighbourhood: ['Restaurants', '10', 'Shopping', '10', 'Transit friendly', '9', 'Daycares', '9', 'Cafes', '9', 'Greenery', '8', 'Historic', '7', 'Quiet', '5', 'Car friendly', '5']
Number of neighborhood indicators:  16
UNITED: [('Elementary Schools', '10'), ('Restaurants', '10'), ('High Schools', '10'), ('Shopping', '10'), ('Pedestrian friendly', '10'), ('Transit friendly', '9'), ('Parks', '10'), ('Daycares', '9'), ('Groceries', '10'), ('Cafes', '9'), ('Vibrant', '10'), ('Greenery', '8'), ('Cycling friendly', '10'), ('Historic', '7'), ('Nightlife', '10'), ('Quiet', '5')]
SET: {('Transi

Page: 13
Start scraping new page...
DESCRIPTION: ['11 rooms', '4 bedrooms (1 in basement)', '2 bathrooms and 1 powder room', 'Building style', 'Two or more storey, Detached', 'Year built', '1967', 'Lot area', '22,500 sqft', 'Parking (total)', 'Garage (2)', 'Pool', 'Inground', 'Fireplace/Stove', 'Wood fireplace', 'Additional features', 'Basement 6 feet or +', '3']
Top neighbourhood: ['Quiet', '10', 'Car friendly', '10', 'Greenery', '9', 'Parks', '5', 'Elementary Schools', '3', 'Groceries', '3', 'Pedestrian friendly', '2', 'Cafes', '2']
Buttom neighbourhood: ['Restaurants', '2', 'Shopping', '2', 'Transit friendly', '1', 'High Schools', '1', 'Daycares', '1', 'Vibrant', '1', 'Historic', '0', 'Nightlife', '0']
Number of neighborhood indicators:  16
UNITED: [('Quiet', '10'), ('Restaurants', '2'), ('Car friendly', '10'), ('Shopping', '2'), ('Greenery', '9'), ('Transit friendly', '1'), ('Parks', '5'), ('High Schools', '1'), ('Elementary Schools', '3'), ('Daycares', '1'), ('Groceries', '3'), ('

...

Page: 82
Start scraping new page...
Waiting for new DOM...
DESCRIPTION: ['11 rooms', '4 bedrooms (1 in basement)', '1 bathroom', 'Building style', 'Bungalow, Detached', 'Year built', '1977', 'Lot area', '8,400 sqft', 'Parking (total)', 'Carport (1), Driveway (4)', 'Fireplace/Stove', 'Wood stove', 'Additional features', 'Basement 6 feet or +', '20']
Top neighbourhood: ['Quiet', '10', 'Car friendly', '9', 'Elementary Schools', '7', 'Daycares', '5', 'Restaurants', '4', 'Shopping', '4', 'Pedestrian friendly', '3', 'Groceries', '3']
Buttom neighbourhood: ['Restaurants', '4', 'Shopping', '4', 'Pedestrian friendly', '3', 'Groceries', '3', 'Nightlife', '3', 'Cafes', '3', 'High Schools', '1', 'Vibrant', '1']
Number of neighborhood indicators:  12
UNITED: [('Quiet', '10'), ('Restaurants', '4'), ('Car friendly', '9'), ('Shopping', '4'), ('Elementary Schools', '7'), ('Pedestrian friendly', '3'), ('Daycares', '5'), ('Groceries', '3'), ('Restaurants', '4'), ('Nightlife', '3'), ('Shopping', '4'), ('C

Page: 85
Start scraping new page...
Waiting for new DOM...
DESCRIPTION: ['13 rooms', '4 bedrooms (1 in basement)', '2 bathrooms', 'Building style', 'Bungalow, Detached', 'Year built', '1986', 'Lot area', '5,382 sqft', 'Parking (total)', 'Driveway (4)', 'Fireplace/Stove', 'Fireplace - Other, Wood stove', 'Additional features', 'Basement 6 feet or +', '20']
Top neighbourhood: ['Quiet', '8', 'High Schools', '7', 'Car friendly', '7', 'Elementary Schools', '6', 'Parks', '6', 'Greenery', '5', 'Transit friendly', '4', 'Daycares', '4']
Buttom neighbourhood: ['Pedestrian friendly', '4', 'Groceries', '4', 'Restaurants', '4', 'Shopping', '4', 'Cafes', '3', 'Vibrant', '2', 'Nightlife', '2', 'Historic', '0']
Number of neighborhood indicators:  16
UNITED: [('Quiet', '8'), ('Pedestrian friendly', '4'), ('High Schools', '7'), ('Groceries', '4'), ('Car friendly', '7'), ('Restaurants', '4'), ('Elementary Schools', '6'), ('Shopping', '4'), ('Parks', '6'), ('Cafes', '3'), ('Greenery', '5'), ('Vibrant', '2

Page: 88
Start scraping new page...
Waiting for new DOM...
DESCRIPTION: ['8 rooms', '2 bedrooms', '2 bathrooms', 'Condominium type', 'Divided', 'Net area', '1,193 sqft', 'Year built', '1988', 'Parking (total)', 'Garage (1)', 'Additional features', 'Elevator', '94']
Top neighbourhood: ['Transit friendly', '10', 'High Schools', '10', 'Daycares', '10', 'Pedestrian friendly', '10', 'Parks', '10', 'Groceries', '10', 'Vibrant', '10', 'Nightlife', '10']
Buttom neighbourhood: ['Cafes', '10', 'Restaurants', '10', 'Shopping', '10', 'Elementary Schools', '9', 'Historic', '7', 'Greenery', '6', 'Car friendly', '4', 'Cycling friendly', '4', 'Quiet', '0']
Number of neighborhood indicators:  16
UNITED: [('Transit friendly', '10'), ('Cafes', '10'), ('High Schools', '10'), ('Restaurants', '10'), ('Daycares', '10'), ('Shopping', '10'), ('Pedestrian friendly', '10'), ('Elementary Schools', '9'), ('Parks', '10'), ('Historic', '7'), ('Groceries', '10'), ('Greenery', '6'), ('Vibrant', '10'), ('Car friendly',

Page: 91
Start scraping new page...
Waiting for new DOM...
Waiting for new DOM...
DESCRIPTION: ['Use of property', 'Residential only', 'Building style', 'Detached', 'Year built', '1961', 'Building area (at ground level)', '840 sqft', 'Lot area', '5,927 sqft', 'Parking (total)', 'Driveway (3), Garage (2)', 'Number of units', 'Residential (3)', 'Residential units', '2 x 3 ½, 1 x 4 ½', 'Main unit', '3 rooms, 1 bedroom, 1 bathroom', 'Potential gross revenue', '$27,300', 'Additional features', 'Basement 6 feet or +', '49']


IndexError: list index out of range

In [ ]:
centris.data.info()

In [442]:
centris.data.describe(include="all")

,title,address,price,lat,long,descriptions,neighbourhood_indicators,demographics
count,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [443]:
# # Elements from top of scrollable list
#     neighbourhood_top = wait_for_xpath(\
#                             "//div[@class='ll-list ps ps--active-y']",\
#                             old_DOM)
    
#     # DEBUGGING-----------------------------------
#     print("FIRST:", [x.text for x in centris.driver.find_elements_by_class_name("ll-header")])
#     for button in centris.driver.find_elements_by_class_name("ll-header"):
#         time.sleep(1)
#         button.click()
#     # DEBUGGING-----------------------------------
    
#     # Scroll to buttom of list to load remaining elements
#     scrollable_bar = centris.driver.find_element_by_xpath(\
#                                             "//div[@class='ps__thumb-y']")
#     scrollable_bar.send_keys(Keys.PAGE_DOWN)
    
#     # Elements from buttom of scrollable list
#     neighbourhood_buttom = wait_for_xpath(\
#                             "//div[@class='ll-list ps ps--active-y']",\
#                             old_DOM_buttom)
    
#     # LOGGING----------------------
#     print("Top neighbourhood:", [x.text for x in neighbourhood_top])
#     print("Buttom neighbourhood:", [x.text for x in neighbourhood_buttom])

In [524]:
# neighbourhood_buttom = centris.driver.find_elements_by_xpath(\
#                              "//div[@class='ll-list ps ps--active-y']")
neighbourhood_buttom = centris.driver.find_elements_by_xpath("//div[@class='ll-list']")
#last_button = neighbourhood_buttom[-1]
print("ONE:", [x.text for x in neighbourhood_buttom])
# Scroll to buttom of list to load remaining elements
scrollable_bar = centris.driver.find_element_by_xpath(\
                                        "//div[@class='ps__thumb-y']")
scrollable_bar.send_keys(Keys.PAGE_DOWN)
scrollable_bar.click()
neighbourhood_buttom = centris.driver.find_elements_by_class_name("ll-list")
#last_button = neighbourhood_buttom[-1]
neighbourhood_buttom[0].text

ONE: []


ElementClickInterceptedException: Message: element click intercepted: Element <div class="ps__thumb-y focus-visible" tabindex="0" style="top: 174px; height: 244px;" data-focus-visible-added=""></div> is not clickable at point (303, 40). Other element would receive the click: <nav class="navbar navbar-expand fixed-top">...</nav>
  (Session info: chrome=84.0.4147.105)


In [515]:
# Scroll to buttom of list to load remaining elements
scrollable_bar = centris.driver.find_element_by_xpath(\
                                        "//div[@class='ps__thumb-y']")
scrollable_bar.send_keys(Keys.PAGE_DOWN)
#scrollable_bar.click()

In [477]:
last_button.click()

In [504]:
scrollable_bar.click()

In [600]:
l=[("A",2), ("A",3), ("B", 3),("A",2),("A",2),("A",3)]

In [602]:
print(l)
print(set(l))

[('A', 2), ('A', 3), ('B', 3), ('A', 2), ('A', 2), ('A', 3)]
{('B', 3), ('A', 2), ('A', 3)}


In [625]:
df = pd.DataFrame()

""


In [ ]:
df.append()